In [ ]:
import pandas as pd
import sqlite3
from prophet import Prophet # type: ignore

In [ ]:
conn = sqlite3.connect('./data/trader_data.db')

query = """SELECT 
                 ticker
                ,ROUND(close, 2) as Close
                ,DATE(Date) AS Date
                ,GICSSector
                ,GICSSectorSub
                 CompanyName
            FROM equity 
            WHERE (1=1)
                AND Date > '2024-01-01'
                AND close IS NOT NULL"""

data = pd.read_sql_query(query, conn)

ticker_list_df = pd.read_sql_query("SELECT DISTINCT ticker FROM equity", conn)

In [ ]:
ticker_list = ticker_list_df['Ticker'].to_list()

final = pd.DataFrame()

total = len(ticker_list)

for num, ticker in enumerate(ticker_list):
    
    print(num / total)
    
    stock_data = data[data['Ticker'] == ticker][['Date', 'Close']].reset_index(drop = True)
    
    stock_data.columns = ['ds', 'y']
    
    m = Prophet()
    
    m.fit(stock_data)
    
    future = m.make_future_dataframe(periods=14)
    
    forecast = m.predict(future)
    
    forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(14)
    
    forecast['ticker'] = ticker
    
    final = pd.concat([forecast, final])
